In [91]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel, TrainingArguments, Trainer, DataCollatorForTokenClassification
import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
import json
import re
# from transformers import AutoModelForTokenClassification


In [135]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
special = {'additional_special_tokens': ["<a>", "</a>"]}
tokenizer.add_special_tokens(special)
data_collator = DataCollatorForTokenClassification(tokenizer)
model = BertModel.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [136]:
# need to follow https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

In [178]:
LABEL_MAP = {0: "O", 1: "E"}

In [179]:
class MyDataset(Dataset):
    def __init__(self, jsonl_file):
        """
        Args:
            tsv_file (string): Path to the csv file with annotations.
        """
        with open(jsonl_file, encoding='utf-8') as f:
            lines = f.read()
            self.data = json.loads(lines)
            
#         for datum in self.data:
#             sent = ""
#             label = []
#             datum['text'] = "Az <a>1926-os úszó-Európa-bajnokság</a>on 4 × 200 <a>méteres</a> gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. <a>Versenytársai Wanié</a> Rezső, Wanié András és Bárány István voltak."

#             tmp = re.sub("\<\/a\>([^ ])", "</a> \g<1>", datum['text'])
#             tmp = re.sub("\<a\>", "<a> ", tmp)
#             for w in tmp.split():
#                 if '<a>' in w:
#                     label.append('E')
#                     if '</a>' in w:
#                         sent += w[3:-4]
#                     else:
#                         sent += w[3:]
#                 else:
#                     if '</a>' in w:
#                         label.append('E')
#                         sent += w.replace('</a>', " ")
#                     else:
#                         label.append('O')
#                         sent += w
#                 if not sent.endswith(" "):
#                     sent += " "
#             datum["text"] = sent
#             datum["label"] = label
        self.start_id = tokenizer("<a>")['input_ids'][1]
        self.end_id = tokenizer("</a>")['input_ids'][1]
            

    def __len__(self):
        return len(self.data)
    
    def tokenize_and_align_labels(self, example):
        tokenized_inputs = tokenizer(example["text"], truncation=True)
        labels = []
        new_input_ids = []
        new_token_type_ids = []
        new_attn_mask = []
        
        index = 0
        answer_state = False

        for i, token_type, attn in zip(
            tokenized_inputs["input_ids"], 
            tokenized_inputs["token_type_ids"], 
            tokenized_inputs["attention_mask"]
        ):
            if i == self.start_id:
                answer_state = True
            elif i == self.end_id:
                answer_state = False
            else:
                new_input_ids.append(i)
                new_token_type_ids.append(token_type)
                new_attn_mask.append(attn)
                if answer_state == True:
                    labels.append(1)
                else:
                    labels.append(0)
        assert answer_state == False # ensure no unenclosed answers
        tokenized_inputs["input_ids"] = new_input_ids
        tokenized_inputs["token_type_ids"] = new_token_type_ids
        tokenized_inputs["attention_mask"] = new_attn_mask
        tokenized_inputs["label"] = labels
        return tokenized_inputs



    def __getitem__(self, idx):
        # probably want to do this as a preprocessing step so it's not called at every getitem?
        tokenized = self.tokenize_and_align_labels(self.data[idx])
        print(tokenizer.convert_ids_to_tokens(tokenized["input_ids"]))

        return tokenized

In [180]:
dev = MyDataset("data/dev.jsonl")

In [181]:
dev[0]


['[CLS]', 'Az', '1926', '-', 'os', 'ús', '##zó', '-', 'Európa', '-', 'bajnokság', 'on', '4', '×', '200', 'méteres', 'g', '##yor', '##s', '##ús', '##zás', 'vált', '##óban', 'ez', '##üst', '##érmes', 'lett', '.', 'Vers', '##eny', '##tár', '##sai', 'Bit', '##ske', '##y', 'Zoltán', ',', 'Wan', '##ié', 'András', 'és', 'Bár', '##ány', 'István', 'voltak', '.', 'A', 'következő', ',', '1927', '-', 'es', 'ús', '##zó', '-', 'Európa', '-', 'bajnokság', '##on', 'szintén', '4', '×', '200', 'méteres', 'g', '##yor', '##s', '##ús', '##zás', 'vált', '##óban', 'bronz', '##érmes', 'lett', '.', 'Vers', '##eny', '##tár', '##sai', 'Wan', '##ié', 'Re', '##zs', '##ő', ',', 'Wan', '##ié', 'András', 'és', 'Bár', '##ány', 'István', 'voltak', '.', '[SEP]']


{'input_ids': [101, 11122, 11472, 118, 10427, 36318, 16993, 118, 33663, 118, 62815, 10135, 125, 247, 10777, 96856, 175, 26101, 10107, 25911, 34304, 34229, 53948, 13112, 61785, 88831, 14852, 119, 46744, 33189, 38454, 32912, 55568, 11275, 10157, 47717, 117, 47426, 43668, 43144, 10256, 72547, 19208, 22348, 24654, 119, 138, 30633, 117, 11442, 118, 10196, 36318, 16993, 118, 33663, 118, 62815, 10263, 36506, 125, 247, 10777, 96856, 175, 26101, 10107, 25911, 34304, 34229, 53948, 77663, 88831, 14852, 119, 46744, 33189, 38454, 32912, 47426, 43668, 20304, 47799, 12516, 117, 47426, 43668, 43144, 10256, 72547, 19208, 22348, 24654, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [174]:
# tokenized_input = dev[1]
# print(tokenized_input["input_ids"])
# print(len(tokenized_input["input_ids"]))
# print(len(tokenized_input["label"]))

# print(tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"]))

['[CLS]', 'A', 'per', '##ui', 'válogatott', 'tagja', '##ként', 'részt', 'vett', 'az', '1930', '-', 'as', 'világbajnokságon', 'illetve', 'az', '1927', '-', 'es', 'és', 'az', '1929', '-', 'es', 'Dél', '-', 'amerikai', 'bajnokság', '##on', '.', '[SEP]']
[101, 138, 10178, 11990, 43826, 27103, 16713, 27628, 30620, 10360, 11028, 118, 10146, 100644, 21415, 10360, 11442, 118, 10196, 10256, 10360, 11336, 118, 10196, 58403, 118, 25489, 62815, 10263, 119, 102]
31
31
['[CLS]', 'A', 'per', '##ui', 'válogatott', 'tagja', '##ként', 'részt', 'vett', 'az', '1930', '-', 'as', 'világbajnokságon', 'illetve', 'az', '1927', '-', 'es', 'és', 'az', '1929', '-', 'es', 'Dél', '-', 'amerikai', 'bajnokság', '##on', '.', '[SEP]']


In [182]:
data_collator([dev[0], dev[1]])

['[CLS]', 'Az', '1926', '-', 'os', 'ús', '##zó', '-', 'Európa', '-', 'bajnokság', 'on', '4', '×', '200', 'méteres', 'g', '##yor', '##s', '##ús', '##zás', 'vált', '##óban', 'ez', '##üst', '##érmes', 'lett', '.', 'Vers', '##eny', '##tár', '##sai', 'Bit', '##ske', '##y', 'Zoltán', ',', 'Wan', '##ié', 'András', 'és', 'Bár', '##ány', 'István', 'voltak', '.', 'A', 'következő', ',', '1927', '-', 'es', 'ús', '##zó', '-', 'Európa', '-', 'bajnokság', '##on', 'szintén', '4', '×', '200', 'méteres', 'g', '##yor', '##s', '##ús', '##zás', 'vált', '##óban', 'bronz', '##érmes', 'lett', '.', 'Vers', '##eny', '##tár', '##sai', 'Wan', '##ié', 'Re', '##zs', '##ő', ',', 'Wan', '##ié', 'András', 'és', 'Bár', '##ány', 'István', 'voltak', '.', '[SEP]']
['[CLS]', 'A', 'per', '##ui', 'válogatott', 'tagja', '##ként', 'részt', 'vett', 'az', '1930', '-', 'as', 'világbajnokságon', 'illetve', 'az', '1927', '-', 'es', 'és', 'az', '1929', '-', 'es', 'Dél', '-', 'amerikai', 'bajnokság', '##on', '.', '[SEP]']


{'input_ids': tensor([[   101,  11122,  11472,    118,  10427,  36318,  16993,    118,  33663,
             118,  62815,  10135,    125,    247,  10777,  96856,    175,  26101,
           10107,  25911,  34304,  34229,  53948,  13112,  61785,  88831,  14852,
             119,  46744,  33189,  38454,  32912,  55568,  11275,  10157,  47717,
             117,  47426,  43668,  43144,  10256,  72547,  19208,  22348,  24654,
             119,    138,  30633,    117,  11442,    118,  10196,  36318,  16993,
             118,  33663,    118,  62815,  10263,  36506,    125,    247,  10777,
           96856,    175,  26101,  10107,  25911,  34304,  34229,  53948,  77663,
           88831,  14852,    119,  46744,  33189,  38454,  32912,  47426,  43668,
           20304,  47799,  12516,    117,  47426,  43668,  43144,  10256,  72547,
           19208,  22348,  24654,    119,    102],
         [   101,    138,  10178,  11990,  43826,  27103,  16713,  27628,  30620,
           10360,  11028,    118, 

In [34]:
# t =  dev.data[0]["text"]
# print(t)
# prev_ind = 0
# newtext = ""
# for i in re.finditer('\<a\>(.*)\<\/a\>', dev.data[0]["text"]):
#     print(i.start())
#     print(i.end())
#     print(i.group(1))
#     newtext += t[prev_ind:i.start()] + i.group(1) + " "
# newtext += t[i.end():]
# print(newtext)

Az <a>1926-os úszó-Európa-bajnokság</a>on 4 × 200 méteres gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. Versenytársai Wanié Rezső, Wanié András és Bárány István voltak.
3
39
1926-os úszó-Európa-bajnokság
Az 1926-os úszó-Európa-bajnokság on 4 × 200 méteres gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. Versenytársai Wanié Rezső, Wanié András és Bárány István voltak.


In [13]:
item, lang, title, inlink_title, context, para = dev.data.iloc[0]

tokenizer([title, item], [item, item])

{'input_ids': [[101, 10175, 118, 10196, 36318, 16993, 118, 33663, 118, 62815, 102, 154, 68073, 10929, 74178, 11305, 102], [101, 154, 68073, 10929, 74178, 11305, 102, 154, 68073, 10929, 74178, 11305, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [77]:
test = "Az <a>1926-os úszó-Európa-bajnokság</a>on 4 × 200 <a>méteres</a> gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. <a>Versenytársai Wanié</a> Rezső, Wanié András és Bárány István voltak."
sent = ""
label = []
test = re.sub("\<\/a\>([^ ])", "</a> \g<1>", test)
for w in test.split():
    if '<a>' in w:
        label.append('E')
        if '</a>' in w:
            sent += w[3:-4]
        else:
            sent += w[3:]
    else:
        if w.endswith("</a>"):
            label.append('E')
            sent += w[:-4]
        else:
            label.append('O')
            sent += w
    if not sent.endswith(" "):
        sent += " "

print(test)
print(len(test.split()))
print(sent)
print(len(sent.split()))

print(label)

Az <a>1926-os úszó-Európa-bajnokság</a> on 4 × 200 <a>méteres</a> gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. <a>Versenytársai Wanié</a> Rezső, Wanié András és Bárány István voltak.
43
Az 1926-os úszó-Európa-bajnokság on 4 × 200 méteres gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. Versenytársai Wanié Rezső, Wanié András és Bárány István voltak. 
43
['O', 'E', 'E', 'O', 'O', 'O', 'O', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'E', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [60]:
dev.data[0]

{'language': 'hu',
 'text': 'Az 1926-os úszó-Európa-bajnokság on 4 × 200 méteres gyorsúszás váltóban ezüstérmes lett. Versenytársai Bitskey Zoltán, Wanié András és Bárány István voltak. A következő, 1927-es úszó-Európa-bajnokságon szintén 4 × 200 méteres gyorsúszás váltóban bronzérmes lett. Versenytársai Wanié Rezső, Wanié András és Bárány István voltak. ',
 'label': ['O',
  'E',
  'E',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [10]:
def preprocess(item):
    

SyntaxError: unexpected EOF while parsing (3865171293.py, line 2)